In [ ]:
!pip install concrete-ml==1.9.0 pandas scikit-learn xgboost joblib networkx

#Phát hiện gian lận thẻ tín dụng bảo mật dựa trên XGBoost và đặc trưng đồ thị trong môi trường mã hóa FHE – Mô phỏng bằng Concrete ML

#0. Chuẩn bị
# 🧩 Giới thiệu

Trong bài thực hành này, chúng tôi mô phỏng **pipeline inference bảo mật (privacy-preserving inference pipeline)**  
cho bài toán **Credit Card Fraud Detection** bằng cách kết hợp **XGBoost** và **Graph Features**,  
triển khai trong môi trường **Fully Homomorphic Encryption (FHE)** thông qua thư viện **Concrete ML** của *Zama*.

---

## Lưu ý
- Dùng với concrete-ml ver <1.5
- Dùng trong môi trường Linux

## 🎯 Mục tiêu

- Hiểu quy trình huấn luyện mô hình trong dạng **plaintext**, sau đó **biên dịch sang FHE-compatible model**.  
- Mô phỏng 3 vai trò chính trong giao thức 3PervPPML (Lưu ý chỉ là mô phỏng cách hoạt động)
  - 🧠 **Analyst** – huấn luyện mô hình, sinh bộ khóa FHE.  
  - 🧾 **User** – xây dựng đồ thị giao dịch và mã hóa dữ liệu.  
  - ☁️ **CSP (Cloud Service Provider)** – thực thi **encrypted inference**.  

---

## 💾 Dữ liệu

Nguồn dữ liệu: **Credit Card Fraud Detection Dataset** từ Kaggle  
🔗 [https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud)

**Mô tả:**
- 284,807 bản ghi giao dịch (`transactions`)  
- 30 thuộc tính (`features`):  
  - 28 đặc trưng ẩn danh (`V1`–`V28`) sinh bằng PCA  
  - `Amount` – số tiền giao dịch  
  - `Time` – thời điểm tương đối  
  - `Class` – nhãn (0 = hợp lệ, 1 = gian lận)  

---

## ⚙️ Quy trình mô phỏng pipeline

Toàn bộ quy trình được chạy **trên cùng một máy (Colab)**,  
trong đó các vai trò Analyst – User – CSP được mô phỏng bằng các bước tuần tự.

### Setup
#### 🧭 1. Analyst – Huấn luyện & chuẩn bị mô hình
- Đọc dữ liệu, chia `train/test`.  
Nguồn dữ liệu: Credit Card Fraud Detection Dataset từ Kaggle
🔗 https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
- Huấn luyện mô hình **XGBoostClassifier** trên plaintext.  
- Biên dịch mô hình sang **FHE-compatible model** bằng **Concrete ML**.  
- Sinh khóa và lưu key_eval

#### 👤 2. User – Xử lý & mã hóa dữ liệu
- Load Client
- Load dữ liệu
- Mã hóa đặc trưng bằng `public key (pk)`.
- Lưu lại để chuyển cho CPS

#### ☁️ 3. CSP – Thực thi inference mã hóa
- Load Server từ Analyst
- Load ciphertext từ User.  
- Load evaluation keys
- Thực thi `model.server_run()` trên ciphertext (Concrete ML runtime).  

#### 🔐 4. Analyst – Giải mã & đánh giá
- Load client
- Load result từ CPS
- Giải mã kết quả bằng `secret key (sk)`.  
- Chuyển kết quả decryption sang label

---

## 🧠 Ý nghĩa

Thực hành này giúp:
- Làm quen với **Concrete ML API**: compile, encrypt, decrypt, inference.  
- Nhận thức rõ **trade-off giữa Privacy và Performance**.  
- Tạo nền tảng cho hướng mở rộng: **FHE cho GNN hoặc Federated Learning**.

---


#### 🧭 1. Analyst – Huấn luyện & chuẩn bị mô hình
- Đọc dữ liệu, chia `train/test`.  
Nguồn dữ liệu: Credit Card Fraud Detection Dataset từ Kaggle
🔗 https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
- Huấn luyện mô hình **XGBoostClassifier** trên plaintext.  
- Biên dịch mô hình sang **FHE-compatible model** bằng **Concrete ML**.  
- Sinh khóa và lưu key_eval

In [ ]:
# Analyst cell
import os, joblib, numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier as XGBPlain

# Concrete ML imports
from concrete.ml.sklearn import XGBClassifier as CML_XGB
from concrete.ml.deployment import FHEModelDev, FHEModelClient  # deployment helpers

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


##1.1 Đọc dữ liệu, chia train/test.
Nguồn dữ liệu: Credit Card Fraud Detection Dataset từ Kaggle 🔗 https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud

In [ ]:
# 0. Read dataset
url = "https://raw.githubusercontent.com/shalakasaraogi/credit-card-fraud-detection/main/csv%20files/creditcard.csv"
df = pd.read_csv(url)
X = df.drop("Class", axis=1)
y = df["Class"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
# 1) Preprocessing save scaler
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

folder_path = "/content/drive/MyDrive/Colab Notebooks/XGBoost-FHE-ConcreteML-Phát hiện gian lận thẻ tín dụng /Data/Analyst/models"
os.makedirs(folder_path, exist_ok=True)
folder_path = os.path.join(folder_path, "scaler.joblib")
joblib.dump(scaler, folder_path)

['/content/drive/MyDrive/Colab Notebooks/XGBoost-FHE-ConcreteML-Phát hiện gian lận thẻ tín dụng /Data/Analyst/models/scaler.joblib']

##1.2 Huấn luyện mô hình **XGBoostClassifier** trên plaintext.  


In [ ]:
plain = XGBPlain(n_estimators=10, max_depth=3, use_label_encoder=False, eval_metric="logloss", random_state=42)
plain.fit(X_train_sc, y_train)

folder_path = "/content/drive/MyDrive/Colab Notebooks/XGBoost-FHE-ConcreteML-Phát hiện gian lận thẻ tín dụng /Data/Analyst/models"
os.makedirs(folder_path, exist_ok=True)
folder_path = os.path.join(folder_path, "plain_model.joblib")
joblib.dump(plain, folder_path)

['/content/drive/MyDrive/Colab Notebooks/XGBoost-FHE-ConcreteML-Phát hiện gian lận thẻ tín dụng /Data/Analyst/models/plain_model.joblib']

##1.3 Biên dịch mô hình sang **FHE-compatible model** bằng **Concrete ML**.  


In [ ]:
cml_model = CML_XGB(n_estimators=10, max_depth=3, n_bits=8)
cml_model.fit(X_train_sc, y_train)
circuit = cml_model.compile(X_train_sc)

EvaluationKeys(server_keyset=<mlir._mlir_libs._concretelang._compiler.ServerKeyset object at 0x7ebf7b544330>)
✅ evaluation.keys đã được lưu.


In [ ]:
# Save model artifacts using FHEModelDev
fhe_dev = FHEModelDev("/content/drive/MyDrive/Colab Notebooks/XGBoost-FHE-ConcreteML-Phát hiện gian lận thẻ tín dụng /Data/Analyst/fhe_artifacts", model=cml_model)
# This will create `client.zip` and `server.zip` (plus serialized_processing.json etc)
fhe_dev.save()

## 1.4 Sinh khóa và lưu key_eval


In [ ]:
client = circuit.client
client.keygen()
client.save("/content/drive/MyDrive/Colab Notebooks/XGBoost-FHE-ConcreteML-Phát hiện gian lận thẻ tín dụng /Data/Analyst/fhe_client.zip")
clientt=client

eval_key = client.evaluation_keys
print(eval_key)
serialized_eval = eval_key.serialize()
with open("/content/drive/MyDrive/Colab Notebooks/XGBoost-FHE-ConcreteML-Phát hiện gian lận thẻ tín dụng /Data/Analyst/keys/evaluation.keys", "wb") as f:
    f.write(serialized_eval)
print("✅ evaluation.keys đã được lưu.")

- Chuyển server.zip cho User, CPS

## Giải mã kết quả

#### 🔐 4. Analyst – Giải mã & đánh giá
- Load client
- Load result từ CPS
- Giải mã kết quả bằng `secret key (sk)`.  
- Chuyển kết quả decryption sang label

##### 4.1 Load client

In [40]:
from concrete import fhe

path="/content/drive/MyDrive/Colab Notebooks/XGBoost-FHE-ConcreteML-Phát hiện gian lận thẻ tín dụng /Data/Analyst/fhe_client.zip"

client=fhe.Client.load(path)

##### 4.2 Load result từ CPS

In [41]:
import os
from concrete import fhe
from concrete.fhe import Value

path="/content/drive/MyDrive/Colab Notebooks/XGBoost-FHE-ConcreteML-Phát hiện gian lận thẻ tín dụng /Data/CPS/Server/results"
num_samples = 5
encrypted_inputs = []
for i in range(5):
    with open(f"{path}/result_{i+1}.bin", "rb") as f:
      y_encrypted = f.read()
    encrypted_sample = Value.deserialize(y_encrypted)
    encrypted_inputs.append(encrypted_sample)
    print(encrypted_sample)

##### 4.3 Giải mã kết quả bằng `secret key (sk)`.  


In [42]:
decrypted_results = []

for enc_val in encrypted_inputs:
    dec_val = clientt.decrypt(enc_val)
    decrypted_results.append(dec_val)

# In kết quả
for i, val in enumerate(decrypted_results):
    print(f"Sample {i+1}: {val}")


Sample 1: [[[ 32  14  72  62  55  -9  47 -90 -75 -43]]]
Sample 2: [[[ 85  68  72  62  55   6  47 -72 -87 -43]]]
Sample 3: [[[ 32  14   0   0 -73   0   0 -65 -37   0]]]
Sample 4: [[[ 85  68   0   0   0  50   0 -29 -46   0]]]
Sample 5: [[[  0 -23  52  62  43 -70  47  26 -87 -45]]]


##### 4.4 Chuyển kết quả decryption sang label

In [43]:
import numpy as np

decrypted_results = []

for enc_val in encrypted_inputs:
    dec_val = clientt.decrypt(enc_val)  # dec_val là dạng mảng/array
    decrypted_results.append(dec_val)

# Chuyển vector decrypted thành label dự đoán
for i, val in enumerate(decrypted_results):
    val_array = np.array(val).flatten()   # flatten về 1D
    label = np.argmax(val_array)          # lấy index max → label dự đoán
    print(f"Sample {i+1}: Label = {label}")

Sample 1: Label = 2
Sample 2: Label = 0
Sample 3: Label = 0
Sample 4: Label = 0
Sample 5: Label = 3
